In [2]:
import pandas as pd
import numpy as np
import aux.acessos as ac
import aux.aed as aed

In [3]:
#incluir limite

In [4]:
query = """
        with perfil as (
select 
cpf
, id_customer
, ds_account_type
, ds_gender
, ds_region
, ds_occupation
, case
                when round((extract(day from current_date - cast(dt_birth as date))/30.5/12 / 10),0) = 2 then 'a - ate 24 anos'
                when round((extract(day from current_date - cast(dt_birth as date))/30.5/12 / 10),0) = 3 then 'b - 25-34 anos'
                when round((extract(day from current_date - cast(dt_birth as date))/30.5/12 / 10),0) = 4 then 'c - 35-44 anos'
                when round((extract(day from current_date - cast(dt_birth as date))/30.5/12 / 10),0) = 5 then 'd - 45-54 anos'
                when round((extract(day from current_date - cast(dt_birth as date))/30.5/12 / 10),0) = 6 then 'e - 55-64 anos'
                when round((extract(day from current_date - cast(dt_birth as date))/30.5/12 / 10),0) = 7 then 'f - 65-74 anos'
                when round((extract(day from current_date - cast(dt_birth as date))/30.5/12 / 10),0) = 8 then 'g - 75-84 anos'
                else 'h - idade NI' end as pf_faixa_idade    
from (
select
g.*, c.cpf
, rank() over (partition by c.cpf order by c.cd_yearmonth desc) as ordem_registro
from customer_curated_zone.ca_book_growth g 
left join customer_curated_zone.ca_book_cliente c on (g.id_customer = c.id_customer)
) where ordem_registro = 1
)
select *,
l.credit_limit
from platform_sandbox_zone.flavia_sow_apos_churn a
left join perfil p on p.cpf = a.cpf
left join processed_zone_database_customer.customer_credit_analysis l on l.customer_id = p.id_customer
where 
--l.credit_limit > 0 and 
ds_account_type = 'credito'

"""

In [5]:
base = ac.df_athena('flavia-costa', query)

Failed to execute query.
Traceback (most recent call last):
  File "/home/flavia.costa/.local/lib/python3.9/site-packages/pyathena/common.py", line 307, in _execute
    query_id = retry_api_call(
  File "/home/flavia.costa/.local/lib/python3.9/site-packages/pyathena/util.py", line 84, in retry_api_call
    return retry(func, *args, **kwargs)
  File "/home/flavia.costa/.local/lib/python3.9/site-packages/tenacity/__init__.py", line 430, in __call__
    do = self.iter(retry_state=retry_state)
  File "/home/flavia.costa/.local/lib/python3.9/site-packages/tenacity/__init__.py", line 367, in iter
    return fut.result()
  File "/usr/lib/python3.9/concurrent/futures/_base.py", line 433, in result
    return self.__get_result()
  File "/usr/lib/python3.9/concurrent/futures/_base.py", line 389, in __get_result
    raise self._exception
  File "/home/flavia.costa/.local/lib/python3.9/site-packages/tenacity/__init__.py", line 433, in __call__
    result = fn(*args, **kwargs)
  File "/home/flavia.

In [36]:
len(base)

3559172

In [7]:
base = base.loc[:,~base.columns.duplicated()]

In [31]:
base_cluster = base[['utilizado_interno_m1', 'utilizado_mercado_m1',
       'utilizado_interno_m2', 'utilizado_mercado_m2', 'utilizado_interno_m3',
       'utilizado_mercado_m3']]
base_cluster.head()

,utilizado_interno_m1,utilizado_mercado_m1,utilizado_interno_m2,utilizado_mercado_m2,utilizado_interno_m3,utilizado_mercado_m3,cluster
7,1000.0,2965.0,916.0,0.0,0.0,0.0,2
10,80.0,161.0,40.0,0.0,0.0,0.0,25
16,20.0,5718.0,2.0,6913.0,2.0,0.0,25
17,1706.0,0.0,1445.0,0.0,1445.0,0.0,20
36,715.0,4807.0,522.0,5065.0,508.0,5049.0,25


In [32]:
for i in base_cluster.columns:
    base_cluster[i] = base_cluster[i].fillna(0.0)

<ipython-input-32-bf75a68f3d3e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_cluster_1[i] = base_cluster_1[i].fillna(0.0)


In [22]:
for i in base_cluster.columns:
    print(base_cluster[i].sum())  

817326959.0
3235673738.0
809787577.0
2118863605.0
552376436.0
1143799913.0


In [23]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

In [24]:
scaler = StandardScaler()
scaler_model = scaler.fit(base_cluster)
data_input_scaled = scaler_model.transform(base_cluster)

In [26]:
kmeans = KMeans(n_clusters = 25, random_state=0).fit(data_input_scaled)
cluster = kmeans.predict(data_input_scaled)
base['cluster'] = cluster

In [27]:
base['cluster'].value_counts(normalize = True)

25    0.420718
2     0.164264
20    0.161034
1     0.069528
27    0.044386
0     0.026798
9     0.025321
23    0.019011
14    0.013535
26    0.012272
6     0.008936
16    0.008789
24    0.007276
17    0.004706
7     0.004161
19    0.002101
13    0.001760
28    0.001757
4     0.001692
10    0.000878
3     0.000579
18    0.000153
11    0.000127
15    0.000107
21    0.000081
29    0.000016
12    0.000003
22    0.000003
5     0.000003
8     0.000003
Name: cluster, dtype: float64

In [ ]:
base['utilizado_interno_total'] = base['utilizado_interno_m1'] + base['utilizado_interno_m2'] + base['utilizado_interno_m3']
base['utilizado_mercado_total'] = base['utilizado_mercado_m1'] + base['utilizado_mercado_m2'] + base['utilizado_mercado_m3']
base['utilizado_total'] = base['utilizado_mercado_total'] + base['utilizado_interno_total']

In [ ]:
base['cluster'] = np.where(base['utilizado_total'] == 0, 26, base['cluster'])

In [ ]:
base['cluster'].value_counts(normalize = True)

In [28]:
base['cluster'].value_counts()

25    291370
2     113762
20    111525
1      48152
27     30740
0      18559
9      17536
23     13166
14      9374
26      8499
6       6189
16      6087
24      5039
17      3259
7       2882
19      1455
13      1219
28      1217
4       1172
10       608
3        401
18       106
11        88
15        74
21        56
29        11
12         2
22         2
5          2
8          2
Name: cluster, dtype: int64

In [16]:
base['flag_interno_m1'] = np.where(base['utilizado_interno_m1'] > 0, 1,0)
base['flag_interno_m2'] = np.where(base['utilizado_interno_m2'] > 0, 1,0)
base['flag_interno_m3'] = np.where(base['utilizado_interno_m3'] > 0, 1,0)
base['flag_mercado_m1'] = np.where(base['utilizado_mercado_m1'] > 0, 1,0)
base['flag_mercado_m2'] = np.where(base['utilizado_mercado_m2'] > 0, 1,0)
base['flag_mercado_m3'] = np.where(base['utilizado_mercado_m3'] > 0, 1,0)


In [17]:
base.groupby('cluster')['flag_interno_m1', 'flag_mercado_m1'].mean()

<ipython-input-17-d1b4a96b1396>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  base.groupby('cluster')['flag_interno_m1', 'flag_mercado_m1'].mean()


,flag_interno_m1,flag_mercado_m1
cluster,,
0,0.946946,0.797060
1,0.795541,0.974443
2,0.050491,0.061724
3,0.933602,0.865537
4,1.000000,1.000000
5,0.939515,0.887386
6,1.000000,0.600000
7,0.723537,0.613218
8,0.814371,0.982036


In [13]:
# var_cl = ['utilizado_interno_m1', 'utilizado_mercado_m1', 'utilizado_interno_m2', 'utilizado_mercado_m2', 'utilizado_interno_m3', 'utilizado_mercado_m3']
# base.groupby('cluster')[var_cl].mean().to_csv('cluster_final.csv')

In [29]:
infos = ['cluster', 'ds_gender', 'ds_region', 'ds_occupation', 'pf_faixa_idade']

base.groupby(infos).count().to_csv('cluster_final_perfis.csv')

In [26]:
base.groupby('cluster')['utilizado_interno_total', 'utilizado_mercado_total'].mean()

<ipython-input-26-a6eacaa2b07d>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  base.groupby('cluster')['utilizado_interno_total', 'utilizado_mercado_total'].mean()


,utilizado_interno_total,utilizado_mercado_total
cluster,,
0,6845.494724,8.012228e+03
1,3206.684964,5.315388e+03
2,1877.273046,9.768307e+04
3,5689.164499,4.454156e+05
4,25370.903871,2.473537e+04
5,728.766584,3.711509e+04
6,3329.790388,2.131580e+06
7,178.321580,5.880365e+03
8,179658.144792,4.138860e+04


In [27]:
valores_analise = ['utilizado_interno_total', 'utilizado_mercado_total']
aed.tira_outlier(base, valores_analise).groupby('cluster')[valores_analise].mean()

,utilizado_interno_total,utilizado_mercado_total
cluster,,
0,6845.494724,8.012228e+03
1,3206.684964,5.315388e+03
2,4412.572131,9.618711e+04
3,7344.956988,4.438333e+05
4,22492.731605,2.331915e+04
5,2167.008912,3.740754e+04
6,7308.464000,2.087064e+06
7,756.038387,5.471886e+03
9,6858.102953,9.118605e+03


In [28]:
base.groupby('cluster')['utilizado_interno_total', 'utilizado_mercado_total'].median()

<ipython-input-28-3a8f7128bef2>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  base.groupby('cluster')['utilizado_interno_total', 'utilizado_mercado_total'].median()


,utilizado_interno_total,utilizado_mercado_total
cluster,,
0,6623.200,3906.0
1,3220.930,3381.0
2,0.000,91971.0
3,0.000,418485.0
4,24182.730,11416.0
5,0.000,34193.5
6,0.000,1988923.0
7,0.000,4134.0
8,165410.220,13655.5


In [29]:
base.groupby('cluster')['utilizado_interno_total', 'utilizado_mercado_total'].sum()

<ipython-input-29-0bf97582e43a>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  base.groupby('cluster')['utilizado_interno_total', 'utilizado_mercado_total'].sum()


,utilizado_interno_total,utilizado_mercado_total
cluster,,
0,3.596623e+07,3.281007e+07
1,2.051304e+09,1.155517e+09
2,9.335679e+07,4.854751e+09
3,2.138557e+07,1.674317e+09
4,9.093439e+08,5.281496e+08
5,1.175581e+08,5.959868e+09
6,3.429684e+05,2.195527e+08
7,1.628144e+08,4.213434e+09
8,3.449436e+07,4.304414e+06


In [31]:
base.groupby('cluster')['credit_limit'].mean()

cluster
0      6475.460570
1      3116.524684
2      2808.345987
3      2953.443850
4      8721.317267
5      2714.199444
6      1948.625000
7      1380.019281
8              NaN
9      5693.043632
10     6910.433333
11     8527.398071
12     2690.031541
13    11587.666667
14             NaN
15     2572.711111
16     2761.770370
17     2718.222483
18     7443.216567
19     7864.631579
Name: credit_limit, dtype: float64

In [32]:
base.to_pickle('base_clusters_share_of_wallet.pkl')